<a href="https://colab.research.google.com/github/etherealxx/volatile-concentration-localux-colab/blob/main/volatile-concentration-localux_colab_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Model Downloader (try this one)
%cd /content
!git clone https://github.com/etherealxx/volatile-concentration-localux-colab /content/colabtools
!pip install -q gradio
!apt -y install -qq aria2

import os, pickle
import gradio as gr
from IPython.display import clear_output

!python /content/colabtools/cloneall2.py
!python /content/colabtools/choosemodel2.py

if os.path.exists('/content/sortedcolabname.pkl'):
  with open('/content/sortedcolabname.pkl', 'rb') as f:
      sortedcolabname = pickle.load(f)

everycolab = '/content/camendurus/lite'
modelischosen = False
modelchosenmessage = ''
chosencolabname = ''

def modelchosen(chosenmodel):
  global modelischosen
  global modelchosenmessage
  global chosencolabname
  modelchosenmessage = "[1;32mModel " + chosenmodel + " was chosen. It'll be downloaded soon. Continue scroll down and wait for the webui to be loaded."
  textbox_text = f"Model '{chosenmodel} was chosen. It'll be downloaded soon. Continue scroll down and wait for the webui to be loaded."
  chosencolabname = chosenmodel
  modelischosen = True
  return [gr.Button.update(visible=False), gr.Textbox.update(value=textbox_text, visible=True)]

with gr.Blocks() as vclcolab:
    with gr.Column():
      choose = gr.Dropdown(sortedcolabname, label="Choose Your Model")
      confirm = gr.Button("Use This Model", variant="primary", visible=True)
      donetext = gr.Textbox("", label="", visible=False)
      confirm.click(modelchosen, inputs=[choose], outputs=[confirm, donetext])
vclcolab.launch()

while True:
  if modelischosen:
    clear_output(wait=True)
    print(modelchosenmessage)
    print('[0m')
    break

aria2c_lines = []

if chosencolabname:
   colabfilename = chosencolabname + '_webui_colab.ipynb'
   if os.path.exists(os.path.join(everycolab, colabfilename)):
      with open(os.path.join(everycolab, colabfilename), 'r', encoding='utf-8') as f:
          for line in f:
              stripped_line = line.strip()
              if stripped_line.startswith('"!aria2c'):
                  aria2c_lines.append(stripped_line)

if aria2c_lines:
  with open('/content/arialist.pkl', 'wb') as f:
      pickle.dump(aria2c_lines, f)

!python /content/colabtools/runaria2.py

In [ ]:
#@title (original colab code)
from IPython.display import clear_output
%cd /content
!git clone https://github.com/etherealxx/volatile-concentration-localux-colab /content/colabtools
!python /content/colabtools/cloneall.py
!pip install -q gradio 
clear_output(wait=True)
!python /content/colabtools/choosemodel.py

%env TF_CPP_MIN_LOG_LEVEL=1

!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

!apt -y install -qq aria2
!pip install -q torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
!pip install -q xformers==0.0.16 triton==2.0.0 -U
!apt -y install -qq aria2

!git clone https://github.com/etherealxx/volatile-concentration-localux /content/breaktherules
!python /content/breaktherules/breaktherules.py

%cd /content/volatile-concentration-localux
!python /content/colabtools/runaria.py
!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/volatile-concentration-localux/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/volatile-concentration-localux/launch.py
!sed -i -e 's/\"sd_model_checkpoint\"\,/\"sd_model_checkpoint\,sd_vae\,CLIP_stop_at_last_layers\"\,/g' /content/volatile-concentration-localux/modules/shared.py

!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple

In [ ]:
#@title (debug)
import os, math, subprocess, pickle
from IPython.display import clear_output

!apt -y install -qq aria2
clear_output(wait=True)

everycolab = '/content/camendurus/lite'
everycolabname = []
colabnamepair = []
for colabname in os.listdir(everycolab):
  colabnamepruned = colabname.partition('_webui_colab.ipynb')[0]
  everycolabname.append(colabnamepruned)

sortedcolabname = sorted(everycolabname)
totalcolabcount = len(everycolabname)
for i, colabname in enumerate(sortedcolabname):
  halfall = math.ceil(totalcolabcount / 2)
  numberedname = "{} | {}".format(i, colabname.ljust(30))
  if i <= halfall:
    colabnamepair.append(numberedname)
  else:
    rev_index = (i - halfall) - 1
    colabnamepair[rev_index] += "\t" + numberedname

for colabpair in colabnamepair:
  print(colabpair)

choosenumber = input('Choose the number of the model you want: ')
if choosenumber.isdigit() and int(choosenumber) < totalcolabcount:
  chosencolabname = sortedcolabname[int(choosenumber)] + '_webui_colab.ipynb'
  print("Preparing to download model from " + chosencolabname)

aria2c_lines = []
with open(os.path.join(everycolab, chosencolabname), 'r', encoding='utf-8') as f:
    for line in f:
        stripped_line = line.strip()
        if stripped_line.startswith('"!aria2c'):
            aria2c_lines.append(stripped_line)

# print(aria2c_lines)
with open('/content/arialist.pkl', 'wb') as f:
    pickle.dump(aria2c_lines, f)

!python /content/runaria.py